## <center> OSNA PJ2 </center>
### <center> Team 20 - Priyam Shah and WIlliam Bodell </center>
### <center> Project Topic: Evidence-enhanced disinformation detection for Crypto News </center>

In [76]:
#imports
import re
import numpy as np
from sumy.parsers.html import HtmlParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
import nltk
from googlesearch import search
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words
import string
import math
import requests
import urllib
from requests_html import HTML
from requests_html import HTMLSession
import sys
import warnings
warnings.filterwarnings('ignore')

#uncomment below command once to download
# nltk.download('punkt') 
# nltk.download('stopwords')
#also install openssl - pip install -U pyopenssl
LANGUAGE = "english"
SENTENCES_COUNT = 10
np.set_printoptions(threshold=np.inf)

links=[]
evidences=[]


In [77]:
#Part 1: filter functions - lowercase, remove non-alphanumberic character, word count, related words from a crypto bag

def word_count(str):
  """
  word_count function takes in a string and returns integer.
  If the word count in input string is greater than six result is 1 else 0
  """
  
  word_count = len(str.split())
  
  if word_count > 6:
    return 1
  return 0

def word_count_evidences(str):
  """
  word_count function takes in a string and returns integer.
  If the word count in input string is greater than twenty five result is 1 else 0
  """
  
  word_count = len(str.split())
  
  if word_count > 25:
    return 0
  return 1

def crypto_space_word(str):
  """
  crypto_space_word function takes in a string and returns integer.
  If the input string has a single word related to crypto space it returns 1 else 0.
  """
  
  crypto_words_list = ["crypto","cryptos","ethereum","eth","bitcoin","btc","privacy","security","nft","amm","money","cash","coins","swap","chain","digital","currencies","currency","cardano","network","metamask","custodians","tokened"]
  
  input_word_list = str.lower().split()
  
  for i in input_word_list:
    if i in crypto_words_list:
      return 1
  return 0

def fake_news_word_count(str):
  """
  fake_news_word function takes in a string and returns integer.
  If the input string has a single word related to reporting fake news it returns 1 else 0.
  """
  
  fake_news_words_list = ["fake", "hoax", "fraud", "fraudulent", "scam", "false", "bullshit", "vapor","blowout","blowouts","closeouts","closeout","negative","pretending","meaningless","purported"]
  
  input_word_list = str.lower().split()
  count = 0
  for i in input_word_list:
    if i in fake_news_words_list:
        count += 1
        
  return count

def remove_nonalphanumeric_char(str):
  """ remove_nonalphanumeric_char function takes in a string and returns alpha-numeric character string only.
  if the input is 
  
  It would be only fair to conclude the day with a @ShubmanGill
 & @rahultewatia02 image 😉 😉

  output will be 
  it would be only fair to conclude the day with a  shubmangill   rahultewatia02 image
  """
  res = re.sub('[^0-9a-zA-Z\s]+', '',  str)
  return res
    


In [78]:
#Part 2: searching article/tweet within known knowledgebase to collect evidences

def cleanup():
    """ cleanup all the blank links or evidences which are irrelevant
    """
    global links, evidences

    pop_set = set()
    for i in range(len(links)):
        try:
            if links[i] == '':
                pop_set.add(i)
            if evidences[i] == '' or word_count_evidences(evidences[i]):
                pop_set.add(i)
        except:
            pop_set.add(i)
            continue
            
    rev_pop_set = list(pop_set)
    rev_pop_set.sort(reverse=True)

    for val in rev_pop_set:

        links.pop(val)
        evidences.pop(val)

            
        
def summarized_data_from_url(url):
  """ summarized_data_from_url function takes in url and summarizes its parsed content to generate evidence document.
  """
  try:
      parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
      stemmer = Stemmer(LANGUAGE)
      summarizer = Summarizer(stemmer)
      summarizer.stop_words = get_stop_words(LANGUAGE)
      temp_ans =[]
      for sentence in summarizer(parser.document, SENTENCES_COUNT):
          #print(sentence)
          temp_ans.append(str(sentence))
      evidences.append("".join(temp_ans))
  
  except Exception as e:
        print(e)
        evidences.append("")
        return

def get_source(url):
    """Return the source code for the provided URL. 
    """

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)
        
def search_knowledgebase(query):
    """ search over google for top related links to be termed as evidences for our query.
    """
    global links
    
    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.co.uk/search?q=" + query)

    links = list(response.html.absolute_links)
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')

    for url in links[:]:
        if url.startswith(google_domains) or url.endswith('.pdf'):
            links.remove(url)

    for link in links:
        #print(link)
        summarized_data_from_url(link)
    
    cleanup()
    

In [79]:
#Part 3: tf-idf calculation and scoring
    
#Preprocessing
def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = set(stopwords.words('english')) - {"not", "no", "nor"}
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) 
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    return data

processed_text = []

# Extracting Data
def extract_data(evidences,test_article,N):
  for i in range(N):
    processed_text.append(word_tokenize(str(preprocess(evidences[i]))))
  t= document_frequency_calculate(evidences,test_article,N)
  return t

DF = {}
total_vocab_size = len(DF)
total_vocab = [x for x in DF]

# Calculating Document frequency of words
def document_frequency_calculate(evidences,test_article,N):  
  for i in range(N):
      tokens = processed_text[i]
      for w in tokens:
          try:
              DF[w].add(i)
          except:
              DF[w] = {i}

  for i in DF:
      DF[i] = len(DF[i])
      
  global total_vocab_size
  total_vocab_size = len(DF)
  global total_vocab
  total_vocab = [x for x in DF]
  t = tf_idf_calculate(evidences,test_article,N)
  return t

def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

tf_idf = {}

def tf_idf_calculate(evidences,test_article,N):
  doc = 0
  for i in range(N):
      
      tokens = processed_text[i]
      
      counter = Counter(tokens + processed_text[i])
      words_count = len(tokens + processed_text[i])
      
      for token in np.unique(tokens):
          
          tf = counter[token]/words_count
          df = doc_freq(token)
          idf = np.log((N+1)/(df+1))
          
          tf_idf[doc, token] = tf*idf

      doc += 1
  t = cosine_similarity(N, test_article,evidences,N)
  return t

def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

def vectorizing_tfidf(evidences,test_article,N):
  total_vocab_size = len(DF)
  D = np.zeros((N, total_vocab_size))
  for i in tf_idf:
   try:
      ind = total_vocab.index(i[1])
      i0=i[0]
      D[i0][ind] = tf_idf[i]
   except:
     pass
  return D

def gen_vector(tokens,N):
    Q = np.zeros((len(total_vocab)))
    counter = Counter(tokens)
    words_count = len(tokens)
    query_weights = {}
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))
        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q



#Part 4: final results

def cosine_similarity(k,test_article,evidences,N):
    D = vectorizing_tfidf(evidences,test_article,N)
    tokens = word_tokenize(str(test_article))
    d_cosines = []    
    query_vector = gen_vector(tokens,N)
    
    di = 0
    for d in D:
        sim = cosine_sim(query_vector, d)
        fake_word_count = fake_news_word_count(evidences[di])
        if fake_word_count > 0:
            sim = sim / (fake_word_count*10.0)
            
        d_cosines.append(sim)
        di = di + 1

    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Final results~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
    sorted_evidences = [x for _,x in sorted(zip(d_cosines,evidences),reverse=True)]
    sorted_urls = [x for _,x in sorted(zip(d_cosines,links),reverse=True)]
    d_cosines.sort(reverse=True)
    
    data_clean = []
    for cosine_value in range(len(d_cosines)):
        if d_cosines[cosine_value] == 0.0 or math.isnan(d_cosines[cosine_value]):
            data_clean.append(cosine_value)
    
    data_clean.sort(reverse=True)
    
    for i in range(len(data_clean)):
        sorted_evidences.pop(data_clean[i])
        sorted_urls.pop(data_clean[i])
        d_cosines.pop(data_clean[i])
        
    for cosine_value in range(len(d_cosines)):
        print("Evidence ",cosine_value,": ",sorted_evidences[cosine_value],"\nUrl: ",sorted_urls[cosine_value],"\nCosine Value: ",d_cosines[cosine_value],"\n\n")
        
    final_score = round(sum(d_cosines)/N,3)

    return final_score 

def scoring(evidences,test_article):
  N=len(evidences)
  score = extract_data(evidences,test_article,N)
  return score


In [80]:
#Script for single news/article testing
#Driver function and single article test - replace string in test_article with your news/article for testing
if __name__ == "__main__":    
    test_article = "Amazon Token is a permissioned blockchain digital currency proposed by the American social media company Amazon. The project, currenct and transactions are to be managed and cryptographically entrusted to the Amazon Token, a membership organization founded by Amazon and 27 others across payment, technology, telecommunication, online marketplace, venture capital and nonrofits."

    #Part 1: filter functions - lowercase, remove non-alphanumberic character, word count, related words from a crypto bag
    test_article=test_article.lower()
    test_article=remove_nonalphanumeric_char(test_article)
    if not crypto_space_word(test_article):
      print("Text content not from crypto space.")
      sys.exit(0)

    if not word_count(test_article):
      print("Too few words to verify!")
      sys.exit(0)

    print("Query Text : ",test_article,"\n")
    
    #Part 2: searching article/tweet within known knowledgebase to collect evidences
    search_knowledgebase(test_article)
    
    #Part 3: tf-idf calculation and scoring
    #Part 4: final results
    score=scoring(evidences,test_article)
    
    print("Average cosine similarity value for query and evidences is ", score,"\n")
    
    if score > 0.05:
        print("Positive 😆!, Content seems to be supported by evidences")
    elif score<0.05 and score>0.00:
        print("Negative 😞!, Content seems to be opposed by evidences")
    else:
        print("Very low score to predict a judgement 🙈")


Query Text :  amazon token is a permissioned blockchain digital currency proposed by the american social media company amazon the project currenct and transactions are to be managed and cryptographically entrusted to the amazon token a membership organization founded by amazon and 27 others across payment technology telecommunication online marketplace venture capital and nonrofits 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Final results~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Evidence  0 :  LACY CLAY, Missouri BILL POSEY, Florida
DAVID SCOTT, Georgia BLAINE LUETKEMEYER, Missouri
AL GREEN, Texas BILL HUIZENGA, Michigan
EMANUEL CLEAVER, Missouri SEAN P. DUFFY, Wisconsin
ED PERLMUTTER, Colorado STEVE STIVERS, Ohio
JIM A. HIMES, Connecticut ANN WAGNER, Missouri
BILL FOSTER, Illinois ANDY BARR, Kentucky
JOYCE BEATTY, Ohio SCOTT TIPTON, Colorado
DENNY HECK, Washington ROGER WILLIAMS, Texas
JUAN VARGAS, California FRENCH HILL, Arkansas
JOSH GOTTHEIMER, New Jersey TOM EMMER, Minnesota
VICENTE GON